In [1]:
import requests
import pandas

In [2]:
# initial input
account = '106010'
password = 'G122117609'
session = requests.session()
chracter_choose = 'cloud' # could choose "cloud" or "yourself"
ticket_type = 'handling'
search_column='disp_seq'
query_string=''
max_count=None

In [3]:
chracters = {
    'cloud': '20',
    'yourself': account,
}

In [4]:
def login_with_given_session(account, password, session):
    post_url = 'http://202.3.168.17:8080/login_check.jsp'
    post_data = {
        'sess_id': account,
        'sess_password': password,
        'dfForm': 'login.jsp',
    }
    http_response = session.post(url=post_url, data=post_data)
    return http_response

In [5]:
login_with_given_session(account=account, password=password, session=session)

<Response [200]>

In [6]:
def search_on_dashboard_terminal(session, character, search_column, query_string, max_count):
    post_url = 'http://202.3.168.17:8080/Disp/DashBoard_Terminal.jsp?action=search'
    post_data = {
        'recordCount': max_count,
        'searchColumn': search_column,
        'searchCondition': query_string,
        'doSearch': 'Search',
        'character': character,
    }
    http_response = session.post(url=post_url, data=post_data)
    return http_response

In [7]:
search_http_response = search_on_dashboard_terminal(
    session=session,
    character=chracters[chracter_choose],
    search_column=search_column,
    query_string=query_string,
    max_count=max_count
)

In [8]:
def get_dataframe_list_by_reading_html(html):
    return pandas.read_html(html)

In [9]:
ticket_tables = get_dataframe_list_by_reading_html(search_http_response.text)

In [10]:
# take the part we want as a dictionary
ticket_tables = {
    'preordered': ticket_tables[2],
    'assigned': ticket_tables[3],
    'handling': ticket_tables[4],
    'finished': ticket_tables[5],
    'special': ticket_tables[6],
}

In [11]:
def let_the_first_row_be_column_title(dataframe):
    new_dataframe = dataframe[1:]
    new_dataframe.columns = dataframe.iloc[0]
    return new_dataframe

In [12]:
for key, value in ticket_tables.items():
    ticket_tables[key] = let_the_first_row_be_column_title(value)

In [13]:
def get_ticket_detail_with_given_ticket_number(session, chracter, ticket_number):
    post_url = 'http://202.3.168.17:8080/Disp/retriveDetail.jsp'
    post_data = {
        'method': 'get_Disp_DetailCons',
        'Disp_Cons_Seq': chracter,
        'Disp_Grp_Seq': chracter,
        'Disp_Seq': ticket_number,
    }
    http_response = session.post(url=post_url, data=post_data)
    return http_response

In [14]:
df_list = []
for ticket_number in list(ticket_tables[ticket_type]['單號']):
    ticket_detail_response = get_ticket_detail_with_given_ticket_number(session=session, chracter=chracters[chracter_choose], ticket_number=ticket_number)
    df_ticket_detail = get_dataframe_list_by_reading_html(ticket_detail_response.text)[0]
    df_ticket_detail = let_the_first_row_be_column_title(df_ticket_detail)
    df_ticket_detail = df_ticket_detail.assign(單號=ticket_number)
    df_list += [df_ticket_detail]

In [15]:
df_left = pandas.concat(df_list)
df_right = ticket_tables[ticket_type]
db_joined = pandas.merge(df_left, df_right, on='單號', how='outer', suffixes=('', '-細項'))

In [16]:
db_joined

,工務名稱,裝機地址,工務同仁,轉派時間,流程,工期,工務起訖日,待竣時間,單號,客戶名稱,協銷客戶名稱,類別,專案,流程-細項
0,其他,NaN,雲端-楊家瑋,2020-04-27 17:42:27,處理中,3,標:2020/04/28~2020/04/30預:2020/05/15~2020/05/16,NaN,202004270937,中國旺旺控股有限公司,NaN,新增/測試單,李瑞賢,處理中
1,VM服務,NaN,雲端-楊家瑋,2020-04-22 18:11:22,處理中,2,標:2020/04/23~2020/04/24預:2020/05/31~2020/06/01,NaN,202004220892,海博資訊股份有限公司,NaN,新增/測試單,李瑞賢,處理中
2,VM服務,NaN,雲端-林敬智,2020-04-21 11:30:40,處理中,3,標:2020/04/22~2020/04/24預:2020/06/17~2020/06/17,NaN,202004210873,傑思網擎有限公司,NaN,異動/測試單,李瑞賢,處理中
3,VM服務-頻寬,NaN,雲端-林敬智,2020-04-21 11:30:45,處理中,9,標:2020/04/22~2020/05/05預:2020/06/17~2020/06/17,NaN,202004210873,傑思網擎有限公司,NaN,異動/測試單,李瑞賢,處理中
4,VM服務,NaN,雲端-林敬智,2020-04-21 11:30:02,處理中,3,標:2020/04/22~2020/04/24預:2020/05/06~2020/05/06,NaN,202004210872,鈊象電子股份有限公司,NaN,新增/測試單,李瑞賢,處理中
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,VM服務,NaN,雲端-楊家瑋,2019-11-05 13:43:40,處理中,3,標:2019/11/06~2019/11/08預:2020/06/30~2020/07/01,NaN,201911042589,先啟資訊系統股份有限公司,NaN,新增/測試單,李瑞賢,處理中
69,其他,NaN,雲端-楊家瑋,2019-11-05 13:43:59,處理中,3,標:2019/11/06~2019/11/08預:2020/06/30~2020/07/01,NaN,201911042589,先啟資訊系統股份有限公司,NaN,新增/測試單,李瑞賢,處理中
70,VM服務-頻寬,NaN,雲端-楊家瑋,2019-11-05 13:43:52,處理中,9,標:2019/11/06~2019/11/18預:2020/06/30~2020/07/01,NaN,201911042589,先啟資訊系統股份有限公司,NaN,新增/測試單,李瑞賢,處理中
71,VM服務,NaN,雲端-楊家瑋,2019-11-05 13:43:46,處理中,3,標:2019/11/06~2019/11/08預:2020/06/30~2020/07/01,NaN,201911042589,先啟資訊系統股份有限公司,NaN,新增/測試單,李瑞賢,處理中
